In [ ]:
# --- Imports ---
import argparse
import pandas as pd
import requests
import time
import os
import re
import math
from dotenv import load_dotenv

# Load environment variables from a `.env` file (for API keys, URLs)
load_dotenv()

# --- Configurable Constants ---
ANYTHINGLLM_API_URL = "http://localhost:3001/api/v1/openai/chat/completions"
ANYTHINGLLM_MODEL = "removepiidata_openaioss"  # Workspace name in AnythingLLM

BATCH_SIZE = 100

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL = "llama3.2"  # Update if you use a different locally pulled model

# --- Helper Functions ---

def clean_llm_response(response: str) -> str:
    """
    Remove common LLM-generated prefixes such as 'Here is the redacted text:'.
    """
    unwanted_prefixes = [
        "Here is the redacted text:",
        "The redacted version is:",
        "Masked version:",
        "Anonymized output:",
        "Output:",
    ]
    for prefix in unwanted_prefixes:
        if response.strip().lower().startswith(prefix.lower()):
            return response[len(prefix):].strip()
    return response.strip()

def mask_pii_using_ollama(text):
    """
    Use a locally running Ollama LLM to mask PII from the given RTI text.
    Returns redacted text.
    """
    if not isinstance(text, str) or text.strip() == "":
        return text

    prompt = f"""
Mask all PII in the text below. Do not rewrite, explain, or generate anything extra. Just replace PII with tags.

The following text is part of a user-submitted Right to Information (RTI) request. It may contain personal details, dates, names, addresses, ID numbers, or sensitive information.

Your job is to only mask personally identifiable information (PII) by replacing:
- Names → [MASKED_NAME]
- Addresses → [MASKED_ADDRESS]
- Phone Numbers → [MASKED_PHONE_NUMBER]
- Company Names → [MASKED_COMPANY_NAME]
- Dates → [MASKED_DATE]
- Any IDs (PAN, Aadhar, Bill Numbers, EPF, etc.) → [MASKED_ID]

Preserve all section markers like <<TITLE>>, <<USER DESCRIPTION>>, etc.
Do not rewrite or rephrase. Do not include headers or explanations.

{text}
"""

    try:
        response = requests.post(OLLAMA_URL, json={
            "model": MODEL,
            "prompt": prompt,
            "stream": False
        })

        if response.status_code == 200:
            raw_output = response.json().get("response", "")
            return clean_llm_response(raw_output)
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return text
    except Exception as e:
        print(f"Error contacting Ollama: {e}")
        return text

def mask_pii_using_anythingllm(text):
    """
    Use AnythingLLM's API to redact PII from the given text using a workspace.
    """
    if not isinstance(text, str) or text.strip() == "":
        print("Error: Empty or invalid input.")
        return text

    api_key = os.getenv("ANYTHINGLLM_API_KEY")
    if not api_key:
        print("❌ Missing API key. Set ANYTHINGLLM_API_KEY in your environment or .env file.")
        return text

    try:
        response = requests.post(
            ANYTHINGLLM_API_URL,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {api_key}"
            },
            json={
                "model": ANYTHINGLLM_MODEL,
                "messages": [
                    {
                        "role": "user",
                        "content": text
                    }
                ],
                "stream": False
            }
        )

        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"].strip()
        else:
            print(f"❌ API Error {response.status_code}: {response.text}")
            return text
    except Exception as e:
        print(f"❌ Error contacting AnythingLLM: {e}")
        return text

# --- Core Batch Processing Function ---

def mask_pii_rti(input_path, intermediate_csv_path, final_xlsx_path,
                 columns_to_mask, columns_to_save_withoutmasking=[], batch_size=100, delete_csv_after=True):
    """
    Process RTI rows in batches. Mask PII using AnythingLLM.
    Save intermediate results to CSV (safe for crash recovery).
    At the end, convert the CSV to one final Excel file.
    """

    # Load input Excel
    try:
        df = pd.read_excel(input_path)
    except Exception as e:
        print(f"❌ Error reading Excel file: {e}")
        return

    total_rows = len(df)
    total_batches = math.ceil(total_rows / batch_size)
    print(f"📄 Total RTIs to process: {total_rows} (Batch size: {batch_size}, Total batches: {total_batches})")

    overall_start = time.time()

    # Clean existing CSV if present
    if os.path.exists(intermediate_csv_path):
        os.remove(intermediate_csv_path)

    # Process in batches
    for batch_num in range(total_batches):
        batch_start_index = batch_num * batch_size
        batch_end_index = min((batch_num + 1) * batch_size, total_rows)
        batch_df = df.iloc[batch_start_index:batch_end_index].copy()

        print(f"\n🚀 Processing batch {batch_num + 1} ({batch_start_index + 1} to {batch_end_index})")

        batch_output_rows = []

        for index in batch_df.index:
            row_start = time.time()
            rti_number = df.at[index, "Number"] if "Number" in df.columns else f"Row {index}"
            print(f"🔍 Processing RTI: {rti_number}")

            # Combine fields with markers
            combined_text = "\n\n".join([
                f"<<{col.upper()}>>\n{str(df.at[index, col])}"
                for col in columns_to_mask if col in df.columns and pd.notna(df.at[index, col])
            ])

            masked_output = mask_pii_using_anythingllm(combined_text)

            # Parse masked text back into fields
            output_row = {col: df.at[index, col] for col in columns_to_save_withoutmasking}
            for col in columns_to_mask:
                tag = f"<<{col.upper()}>>"
                pattern = re.compile(rf"{re.escape(tag)}\s*(.*?)(?=(?:<<[A-Z _]+>>)|\Z)", re.DOTALL)
                match = pattern.search(masked_output)
                output_row[f"Masked {col}"] = match.group(1).strip() if match else ""

            batch_output_rows.append(output_row)
            print(f"✅ Done with RTI {rti_number} | ⏱️ Time taken: {round(time.time() - row_start, 2)} sec")

        # Save batch to intermediate CSV
        try:
            batch_output_df = pd.DataFrame(batch_output_rows)
            write_header = not os.path.exists(intermediate_csv_path)
            batch_output_df.to_csv(intermediate_csv_path, mode='a', index=False, header=write_header, encoding='utf-8')
            print(f"💾 Appended batch {batch_num + 1} to: {intermediate_csv_path}")
        except Exception as e:
            print(f"❌ Error saving batch {batch_num + 1} to CSV: {e}")

    # Convert CSV to Excel
    try:
        final_df = pd.read_csv(intermediate_csv_path)
        final_dir = os.path.dirname(final_xlsx_path)
        if final_dir:
            os.makedirs(final_dir, exist_ok=True)
        final_df.to_excel(final_xlsx_path, index=False)
        print(f"\n✅ Final Excel file saved to: {final_xlsx_path}")
    except Exception as e:
        print(f"❌ Error converting CSV to Excel: {e}")

    # Optional cleanup
    if delete_csv_after:
        try:
            os.remove(intermediate_csv_path)
            print(f"🧹 Deleted intermediate CSV: {intermediate_csv_path}")
        except Exception as e:
            print(f"⚠️ Could not delete intermediate CSV: {e}")

    total_time = time.time() - overall_start
    print(f"\n⏱️ Total processing time: {round(total_time, 2)} sec")
    print(f"🕒 Avg per RTI: {round(total_time / total_rows, 2)} sec")

# --- Optional Column-wise Redaction Function ---

def redact_excel_file(input_path, output_path, pii_columns):
    """
    Redacts each specified column individually using Ollama (for testing).
    """
    try:
        df = pd.read_excel(input_path)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    for col in pii_columns:
        if col not in df.columns:
            print(f"Column '{col}' not found in Excel. Skipping.")
            continue

        print(f"Masking PII in column: {col}")
        df[col] = df[col].apply(mask_pii_using_ollama)

    try:
        df.to_excel(output_path, index=False)
        print(f"✅ Anonymized Excel saved to: {output_path}")
    except Exception as e:
        print(f"Error saving redacted Excel file: {e}")

# --- Main Execution ---

if __name__ == "__main__":
    # File paths
    input_file = "./test.xlsx"
    output_file = "./test_outputOpenAI.xlsx"
    temp_file = "./mask_pii_rti.csv"

    # Columns to process
    columns_to_mask_pii = ["Title", "User Description", "Drafter Modified Title", "Drafter Modified Description"]
    columns_to_save_withoutmasking = ["Number"]

    # Run PII masking
    mask_pii_rti(
        input_file,
        temp_file,
        output_file,
        columns_to_mask_pii,
        columns_to_save_withoutmasking,
        BATCH_SIZE,
        delete_csv_after=True
    )

    print("✅ Done")